In [1]:
# 2021 07 15 в каб. Б.В. с экологами. Делаю базу для заполнения и рассылку отчетов по вывозу отходов


In [2]:
import os
os.getcwd()
os.chdir('C:\\Reports')
os.getcwd()

'C:\\Reports'

In [3]:
# import os
# try:
# 	os.chdir(os.path.join(os.getcwd(), '\\\\172.25.0.5\\share\\ФинДир\\Python'))
# 	print(os.getcwd())
# except:
# 	pass

In [4]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date, timedelta
import openpyxl
import xlsxwriter
import re

# import pymysql.cursors

import win32com.client as win32
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
from email.mime.multipart import MIMEMultipart


In [5]:
# pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('display.max_rows', 100)

In [6]:
# чтение файла данных

df = pd.read_excel('\\\\XXX.XX.XX.X\\XXXX\\XXX\\учет отходов.xlsx',sheet_name='base', engine='openpyxl', header=0, usecols="A:I")

# временно переключился на копию. Что то не перестал видеть все отходы вывезенные Фрегат-ЮГ


# df.tail()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Дата             773 non-null    datetime64[ns]
 1   Время            0 non-null      float64       
 2   Компания         773 non-null    object        
 3   АвтоГосНом       713 non-null    object        
 4   Вид_отходов      773 non-null    object        
 5   КубМетр          257 non-null    float64       
 6   Тонн             692 non-null    float64       
 7   Класс опасности  773 non-null    int64         
 8   Сумма_руб        127 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 54.5+ KB


C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
df['Дата'] = pd.to_datetime(df['Дата'])

In [8]:
df = df.sort_values(by='Дата')

df.tail()

,Дата,Время,Компания,АвтоГосНом,Вид_отходов,КубМетр,Тонн,Класс опасности,Сумма_руб
758,2021-10-29,NaN,Золотаръ,А474УС/134,строительный мусор,NaN,NaN,4,NaN
757,2021-10-29,NaN,Золотаръ,М937ЕC/34,ОТХОДЫ ЛКМ,NaN,5.52,4,44160.0
771,2021-10-29,NaN,Золотаръ,К387ТУ/134,строительный мусор,NaN,NaN,4,NaN
763,2021-10-29,NaN,Золотаръ,О793ЕЕ/47,строительный мусор,NaN,NaN,4,NaN
772,2021-10-29,NaN,Золотаръ,К387ТУ/134,строительный мусор,NaN,NaN,4,NaN


In [9]:
# проверка сумма
df['КубМетр'].sum()


3047.68

In [10]:
# добавляю столбцы
df['Год'] = df['Дата'].dt.year.astype('Int64').values
df['Месяц'] = df['Дата'].dt.month.astype('Int64').values
df['День'] = df['Дата'].dt.day.astype('Int64').values

# df["ST"] = pd.to_numeric(df["Сумма_руб"]/1000).round(2)

#только год и месяц
df['YM'] = df['Дата'].dt.to_period('M')

In [11]:
df.tail()

,Дата,Время,Компания,АвтоГосНом,Вид_отходов,КубМетр,Тонн,Класс опасности,Сумма_руб,Год,Месяц,День,YM
758,2021-10-29,NaN,Золотаръ,А474УС/134,строительный мусор,NaN,NaN,4,NaN,2021,10,29,2021-10
757,2021-10-29,NaN,Золотаръ,М937ЕC/34,ОТХОДЫ ЛКМ,NaN,5.52,4,44160.0,2021,10,29,2021-10
771,2021-10-29,NaN,Золотаръ,К387ТУ/134,строительный мусор,NaN,NaN,4,NaN,2021,10,29,2021-10
763,2021-10-29,NaN,Золотаръ,О793ЕЕ/47,строительный мусор,NaN,NaN,4,NaN,2021,10,29,2021-10
772,2021-10-29,NaN,Золотаръ,К387ТУ/134,строительный мусор,NaN,NaN,4,NaN,2021,10,29,2021-10


In [12]:
#Сегодня
# now = datetime.today()

# #now.month
# #now.day
# #now.hour
# #now.minute

# # текущий день
# td = now.day

# # текущий месяц
# tm = now.month

# tm

In [13]:
# # нахождение крайней даты через сводную табл. без итогов и reset_index

pt = pd.pivot_table(df, index = ['Дата'], values = ['КубМетр'], aggfunc=np.sum, fill_value='', margins=False)

pt = pt.reset_index()

# pt5.tail()

lrd = pt["Дата"].tail(1)

print(lrd)
print('-------')

#привожу в более менее удобный формат для выдачи в текст заголовка
lrd2 = lrd.astype('str').values

print(lrd2)
print('-------')

118   2021-10-29
Name: Дата, dtype: datetime64[ns]
-------
['2021-10-29']
-------


In [14]:
#сводная табл куб м
pt1 = pd.pivot_table(df, index = ['Компания', 'Вид_отходов'], values = ['КубМетр'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True) #

pt1

КубМетр        \
Год                                                               2020         
Месяц                                                                6     7   
Компания                      Вид_отходов                                      
Золотаръ                      ОТХОДЫ ЛКМ                                       
                              САНЗОНА                                          
                              САНИТАРНАЯ ЗОНА                                  
                              известь                                          
                              продукция органи-ческого синтеза                 
                              строительный мусор                               
                              тара с ЛКМ                                       
                              тара, загряз. нефтепродуктами                    
                              шлам                                             
Комус                         ТБО                                        8.0   
Промтех                       строительный мусор                               
Ситиматик-Волгоград           ТКО                                              
Транском                      известь                                          
                              строительный мусор                               
Управление отходами-Волгоград ТКО                                  1.5   3.0   
Фрегат ЮГ                     известь                                          
                              строительный мусор                               
All                                                                1.5  11.0   

                                                                            \
Год                                                                          
Месяц                                                              8     9   
Компания                      Вид_отходов                                    
Золотаръ                      ОТХОДЫ ЛКМ                                     
                              САНЗОНА                                        
                              САНИТАРНАЯ ЗОНА                                
                              известь                                        
                              продукция органи-ческого синтеза               
                              строительный мусор                             
                              тара с ЛКМ                                     
                              тара, загряз. нефтепродуктами                  
                              шлам                                           
Комус                         ТБО                               32.0  64.0   
Промтех                       строительный мусор                             
Ситиматик-Волгоград           ТКО                                            
Транском                      известь                                        
                              строительный мусор                             
Управление отходами-Волгоград ТКО                                1.5   1.5   
Фрегат ЮГ                     известь                                        
                              строительный мусор                             
All                                                             33.5  65.5   

                                                                            \
Год                                                                          
Месяц                                                             10    11   
Компания                      Вид_отходов                                    
Золотаръ                      ОТХОДЫ ЛКМ                                     
                              САНЗОНА                                        
                              САНИТАРНАЯ ЗОНА                                
                              известь  

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 773 entries, 0 to 772
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Дата             773 non-null    datetime64[ns]
 1   Время            0 non-null      float64       
 2   Компания         773 non-null    object        
 3   АвтоГосНом       713 non-null    object        
 4   Вид_отходов      773 non-null    object        
 5   КубМетр          257 non-null    float64       
 6   Тонн             692 non-null    float64       
 7   Класс опасности  773 non-null    int64         
 8   Сумма_руб        127 non-null    float64       
 9   Год              773 non-null    Int64         
 10  Месяц            773 non-null    Int64         
 11  День             773 non-null    Int64         
 12  YM               773 non-null    period[M]     
dtypes: Int64(3), datetime64[ns](1), float64(4), int64(1), object(3), period[M](1)
memory usage: 86.

In [16]:
#сводная табл тонн
pt2 = pd.pivot_table(df, index = ['Компания', 'Вид_отходов'], values = ['Тонн'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt2

Тонн        \
Год                                                             2020         
Месяц                                                              6     7   
Компания                      Вид_отходов                                    
Золотаръ                      ОТХОДЫ ЛКМ                                     
                              САНЗОНА                                        
                              САНИТАРНАЯ ЗОНА                                
                              известь                                        
                              продукция органи-ческого синтеза               
                              строительный мусор                             
                              тара с ЛКМ                                     
                              тара, загряз. нефтепродуктами                  
                              шлам                                           
Комус                         ТБО                                     4.22   
Промтех                       строительный мусор                             
Ситиматик-Волгоград           ТКО                                            
Транском                      известь                                        
                              строительный мусор                             
Управление отходами-Волгоград ТКО                               0.33  0.66   
Фрегат ЮГ                     известь                                        
                              строительный мусор                             
All                                                             0.33  4.88   

                                                                              \
Год                                                                            
Месяц                                                               8      9   
Компания                      Вид_отходов                                      
Золотаръ                      ОТХОДЫ ЛКМ                                       
                              САНЗОНА                                          
                              САНИТАРНАЯ ЗОНА                                  
                              известь                                          
                              продукция органи-ческого синтеза                 
                              строительный мусор                               
                              тара с ЛКМ                                       
                              тара, загряз. нефтепродуктами                    
                              шлам                                             
Комус                         ТБО                               16.02  33.92   
Промтех                       строительный мусор                               
Ситиматик-Волгоград           ТКО                                              
Транском                      известь                                          
                              строительный мусор                               
Управление отходами-Волгоград ТКО                                0.33   0.33   
Фрегат ЮГ                     известь                                          
                              строительный мусор                               
All                                                             16.35  34.25   

                                                                              \
Год                                                                            
Месяц                                                              10     11   
Компания                      Вид_отходов                                      
Золотаръ                      ОТХОДЫ ЛКМ                                       
                              САНЗОНА                                          
                              САНИТАРНАЯ ЗОНА                                  
                          

In [17]:
#сводная табл тонн
pt21 = pd.pivot_table(df, index = ['Компания'], values = ['Тонн'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt21

Тонн                                           \
Год                            2020                                            
Месяц                             6     7      8      9     10     11     12   
Компания                                                                       
Золотаръ                                                                       
Комус                                4.22  16.02  33.92  15.58  28.42  16.46   
Промтех                                                                        
Ситиматик-Волгоград                                                            
Транском                                                                       
Управление отходами-Волгоград  0.33  0.66   0.33   0.33    1.1   0.66   0.88   
Фрегат ЮГ                                                                      
All                            0.33  4.88  16.35  34.25  16.68  29.08  17.34   

                                                                       \
Год                            2021                                     
Месяц                             1      2      3     5      6      7   
Компания                                                                
Золотаръ                                                         9.38   
Комус                                42.48  12.86   1.6                 
Промтех                                                                 
Ситиматик-Волгоград                                       1.76   1.98   
Транском                                                                
Управление отходами-Волгоград  1.32          0.66  2.86                 
Фрегат ЮГ                                                47.05   37.0   
All                            1.32  42.48  13.52  4.46  48.81  48.36   

                                                                    
Год                                                            All  
Месяц                                8        9       10            
Компания                                                            
Золотаръ                          42.4  1304.44  6661.22   8017.44  
Комус                                                       171.56  
Промтех                                            40.98     40.98  
Ситиматик-Волгоград               1.98     1.98                7.7  
Транском                                 389.32   764.58    1153.9  
Управление отходами-Волгоград                                 9.13  
Фрегат ЮГ                      1340.41           1037.59   2462.05  
All                            1384.79  1695.74  8504.37  11862.76

In [18]:
# Переменные с итогами

total_1 = df['Тонн'].sum().round(1)
total_1

11862.8

In [19]:
df2021 = df[df['Год']==2021]
total_2 = df2021['Тонн'].sum().round(1)
total_2

11743.8

In [20]:
now = datetime.today()
# текущий месяц
tm = now.month

df_tm= df[(df['Год']==2021) & (df['Месяц']== tm)]
total_3 = df_tm['Тонн'].sum().round(1)
total_3


0.0

In [21]:
#табл для вывода крайних 5 записей 

# df5 = df.sort_values(by=['Дата'], ascending=True)
# df5= df.tail(5)
# df5

pt3 = pd.pivot_table(df, index = ['Дата','Компания', 'Вид_отходов'], values = ['КубМетр','Тонн'], aggfunc=np.sum, fill_value='', margins=False) #,'Компания', 'Вид_отходов', 'Класс_опасности'

pt3 = pt3.reset_index()

pt3 = pt3.tail(10)

pt3

,Дата,Компания,Вид_отходов,КубМетр,Тонн
144,2021-10-18,Транском,строительный мусор,0.0,52.52
145,2021-10-18,Фрегат ЮГ,строительный мусор,0.0,174.80
146,2021-10-20,Золотаръ,строительный мусор,0.0,271.12
147,2021-10-21,Золотаръ,ОТХОДЫ ЛКМ,4.0,6.62
148,2021-10-21,Золотаръ,строительный мусор,0.0,246.96
149,2021-10-25,Золотаръ,строительный мусор,0.0,0.00
150,2021-10-26,Золотаръ,строительный мусор,0.0,0.00
151,2021-10-28,Золотаръ,строительный мусор,0.0,0.00
152,2021-10-29,Золотаръ,ОТХОДЫ ЛКМ,0.0,5.52
153,2021-10-29,Золотаръ,строительный мусор,0.0,0.00


In [22]:
def write_to_html_file(df, title='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 90%;
    text-align: left;
    color: #191970;
    vertical-align: middle;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%;
  }

  thead > tr:nth-child(2) th:not(:first-child) {
    text-align: center;
  }
  tbody td {
    text-align: right;
  }



</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(float_format='{:.2f}'.format) # classes='wide', escape=False, float_format='{:.2f}'.format
    result += '''

# <script>

#   function numberWithSpaces(x) {
#     var rounded = Math.floor(Number(x));
#     var parts = rounded.toString().split(".");
#     parts[0] = parts[0].replace(/\B(?=(\d{3})+(?!\d))/g, " ");
#     return parts.join(".");
#   }

#   var items = document.querySelectorAll('tbody td');

#   items.forEach(function(item) {
#     var formattedNumber = numberWithSpaces(item.textContent);
#     if(item.textContent === '') {
#       return;
#     }
#     item.textContent = formattedNumber;
#   });

# </script>

<center>
	<footer>
      <br />
      <br />
      <b>Создал:</b> <i>Смоленышев Олег В.</i>
      <br />
      <a href="mailto:smolenyshev@gtm-one.com">smolenyshev@gtm-one.com</a>
  </footer>
</center>

</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [23]:
#текст для заголовков с датой
text1 = "Промтех. Вывоз отходов в куб.м. Данные из файла эколога Балабан В.С. Отчет на дату: {a}".format(a=lrd2)
text2 = "Промтех. Вывоз отходов в тоннах. Данные из файла эколога Балабан В.С. Отчет на дату: {a}".format(a=lrd2)
text3 = "Промтех. Вывоз отходов. в тоннах. Данные из файла эколога Балабан В.С.: крайние 10 записей. Отчет на дату: {a}".format(a=lrd2)



text2

"Промтех. Вывоз отходов в тоннах. Данные из файла эколога Балабан В.С. Отчет на дату: ['2021-10-29']"

In [24]:
#создание html файла

write_to_html_file(pt1, text1, 'Отходы_куб_метр.html')
write_to_html_file(pt2, text2, 'Отходы_тонн.html')
write_to_html_file(pt21, text2, 'Отходы_тонн2.html')
write_to_html_file(pt3, text3, 'Отходы_10_записей.html')


In [30]:
report1 = write_to_html_file(pt1, text1, 'Отходы_куб_метр.html')

print(report1)

None


In [25]:
#пакеты для работы с почтой
import smtplib
import mimetypes
import email
import email.mime.application
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [26]:
# text = f"Добрый день. \n Передаю для информации отчет о вывозе отходов. Отчет на дату: {lrd2}. \n Вывезено отходов с начала 2021 года: {total_2} тонн. Вывезено отходов за текущий ({tm}) месяц: {total_3} тонн \n Отчет формируется каждый рабочий день в 16:30 на основании данных эколога Балабан В.С. \n\n С уважением, Смоленышев О.В."  #текст внутри пиьма, если нужен

# text

In [35]:
sender_email = 'syslog2@XXX.XXX'


receiver_email = 'smolenyshev@gtm-one.com'

username = 'XXX'
password = 'XXX'
server = smtplib.SMTP('XXX')
subject = f'TEST. Промтех. Вывозе отходов. Отчет на дату: {lrd2} (В дополнение не переданного 14 окт 2021)'                                      #тема пиьма
body = f"Добрый день. \n\n {pt1} \n\n Передаю для информации отчет о вывозе отходов. Отчет на дату: {lrd2}. \n Вывезено отходов с начала 2021 года: {total_2} тонн. \n Вывезено отходов за текущий ({tm}) месяц: {total_3} тонн \n\n Отчет формируется каждый рабочий день в 16:30 на основании данных эколога Балабан В.С. \n\n С уважением, Смоленышев О.В."
dir_path = "C:\\Reports"  #путь к файлам
files = ['Отходы_куб_метр.html', 'Отходы_тонн.html', 'Отходы_10_записей.html']        #список отправляемых файлов.

In [36]:
# Создание составного сообщения и установка заголовка

message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject


# Внесение тела письма
message.attach(MIMEText(body, "plain"))

for f in files:
    file_path = os.path.join(dir_path,f)
    attachment = MIMEApplication(open(file_path, "rb").read(), _subtype="txt")
    attachment.add_header('Content-Disposition', 'attachment', filename=f)
    message.attach(attachment)


# %%
# Отпавляем письмо

server.starttls()
server.ehlo()
server.login(username, password)
server.sendmail(message['From'], message['To'].split(','), message.as_string())
server.quit()
print(receiver_email, files, sep='\n')

smolenyshev@gtm-one.com
['Отходы_куб_метр.html', 'Отходы_тонн.html', 'Отходы_10_записей.html']


In [29]:
#Сохранение всех сводных табл в excel файл
# writer = pd.ExcelWriter('promtex_waste_v4.xlsx', engine='xlsxwriter')

# df.to_excel(writer, sheet_name='df', header = True)
# pt1.to_excel(writer, sheet_name='pt1', header = True)
# pt2.to_excel(writer, sheet_name='pt2', header = True)


writer.save()
writer.close()

C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
